In [2]:
import torch
import pandas as pd
import numpy as np
import torchio as tio
from torch.utils.data import DataLoader
import os
import SimpleITK as sitk

In [3]:
#read metadata, and add columns for additional information
csvPath='/home/sliceruser/labels/clinical_information/marksheet.csv'
df = pd.read_csv(csvPath)
#initializing empty columns
df["reSampledPath"] = ""
df["adc"] = ""
df["cor"] = ""
df["hbv"] = ""
df["sag"] = ""
df["t2w"] = ""
df["anythingInMask"] = False
df

,Unnamed: 0,patient_id,study_id,mri_date,patient_age,psa,psad,prostate_volume,histopath_type,lesion_GS,reSampledPath,adc,cor,hbv,sag,t2w,anythingInMask
0,0,10000,1000000,2019-07-02,74,7.70,NaN,55.0,MRBx,0+0,,,,,,,False
1,1,10001,1000001,2016-05-27,64,8.70,0.09,102.0,NaN,NaN,,,,,,,False
2,2,10002,1000002,2021-04-18,58,4.20,0.06,74.0,NaN,NaN,,,,,,,False
3,3,10003,1000003,2019-04-05,72,13.00,NaN,71.5,SysBx,0+0,,,,,,,False
4,4,10004,1000004,2020-10-21,67,8.00,0.10,78.0,SysBx+MRBx,"0+0,0+0",,,,,,,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,1495,11471,1001495,2012-08-25,71,12.50,0.21,62.0,MRBx,"3+4,N/A,3+3",,,,,,,False
1496,1496,11472,1001496,2019-06-28,81,5.28,0.12,44.0,SysBx+MRBx,3+4,,,,,,,False
1497,1497,11473,1001497,2017-09-24,56,29.60,0.34,87.0,MRBx,0+0,,,,,,,False
1498,1498,11474,1001498,2016-05-03,75,12.00,NaN,83.0,MRBx,3+3,,,,,,,False


In [51]:
#create a dictionary of directories where key is the patient_id
rootdir = '/home/sliceruser/data/'
dirDict={}

for subdir, dirs, files in os.walk(rootdir):
    for subdirin, dirsin, filesin in os.walk(subdir):
        lenn= len(filesin)
        if(lenn>0):
            dirDict[subdirin.split("/")[4]]=filesin

labelsFiles=[]
labelsRootDir = '/home/sliceruser/labels/csPCa_lesion_delineations/human_expert/resampled/'
for subdir, dirs, files in os.walk(labelsRootDir):
    labelsFiles=files
labelsFiles

['10589_1000603.nii.gz',
 '10982_1001001.nii.gz',
 '11034_1001054.nii.gz',
 '10075_1000075.nii.gz',
 '10450_1000458.nii.gz',
 '10925_1000942.nii.gz',
 '11131_1001154.nii.gz',
 '10672_1000688.nii.gz',
 '10715_1000731.nii.gz',
 '11070_1001092.nii.gz',
 '10985_1001004.nii.gz',
 '10522_1000532.nii.gz',
 '11328_1001351.nii.gz',
 '10454_1000462.nii.gz',
 '10368_1000374.nii.gz',
 '10014_1000014.nii.gz',
 '10579_1000592.nii.gz',
 '10335_1000341.nii.gz',
 '11337_1001360.nii.gz',
 '11057_1001078.nii.gz',
 '10731_1000747.nii.gz',
 '10020_1000020.nii.gz',
 '11199_1001222.nii.gz',
 '10608_1000622.nii.gz',
 '11347_1001370.nii.gz',
 '10226_1000230.nii.gz',
 '10802_1000818.nii.gz',
 '10533_1000543.nii.gz',
 '11100_1001123.nii.gz',
 '10055_1000055.nii.gz',
 '10296_1000302.nii.gz',
 '11282_1001305.nii.gz',
 '10207_1000211.nii.gz',
 '11388_1001412.nii.gz',
 '11438_1001462.nii.gz',
 '11267_1001290.nii.gz',
 '10269_1000273.nii.gz',
 '10242_1000246.nii.gz',
 '10202_1000206.nii.gz',
 '10467_1000475.nii.gz',


In [4]:

#create a dictionary of directories where key is the patient_id
rootdir = '/home/sliceruser/data/'

dirDict={}
for subdir, dirs, files in os.walk(rootdir):
    for subdirin, dirsin, filesin in os.walk(subdir):
        lenn= len(filesin)
        if(lenn>0):
            dirDict[subdirin.split("/")[4]]=filesin

labelsFiles=[]
labelsRootDir = '/home/sliceruser/labels/csPCa_lesion_delineations/human_expert/resampled/'
for subdir, dirs, files in os.walk(labelsRootDir):
    labelsFiles=files
    
#Constructing functions that when applied to each row will fill the necessary path data
listOfDeficientStudyIds=[]

def findPathh(row,dirDictt,keyWord,rootdir):
    patId=str(row['patient_id'])
    study_id=str(row['patient_id'])
    #first check is such key present
    if(patId in dirDictt ):
        filtered = list(filter(lambda file_name:   (keyWord in file_name and  study_id  in  file_name  ), dirDictt[patId]  ))
        if(len(filtered)>0):
            return os.path.join(rootdir,  patId, filtered[0] )
        else:
            print(f"no {keyWord} in {study_id}")
            listOfDeficientStudyIds.append(study_id)
            return " " 
    listOfDeficientStudyIds.append(study_id)
    return " "
    

def addPathsToDf(dff, dirDictt, keyWord):
    return dff.apply(lambda row : findPathh(row,dirDictt ,keyWord,rootdir )   , axis = 1)

df['t2w'] =addPathsToDf(df,dirDict, 't2w')
df["adc"] = addPathsToDf(df,dirDict, 'adc')
df["cor"] = addPathsToDf(df,dirDict, 'cor')
df["hbv"] = addPathsToDf(df,dirDict, 'hbv')
df["sag"] = addPathsToDf(df,dirDict, 'sag')
#now  resampled labels are in separate directory


def findResampledLabel(row,labelsFiles):
    patId=str(row['patient_id'])
    study_id=str(row['patient_id'])
    filtered = list(filter(lambda file_name:   (study_id  in  file_name  ), labelsFiles ))
    if(len(filtered)>0):
        return os.path.join(labelsRootDir, filtered[0])
    listOfDeficientStudyIds.append(study_id)    
    return " "
    
        
df["reSampledPath"] =  df.apply(lambda row : findResampledLabel(row,labelsFiles )   , axis = 1)  

df



no cor in 10776
no cor in 10941
no cor in 11217
no sag in 10116
no sag in 10691
no sag in 10776
no sag in 10941


,Unnamed: 0,patient_id,study_id,mri_date,patient_age,psa,psad,prostate_volume,histopath_type,lesion_GS,reSampledPath,adc,cor,hbv,sag,t2w,anythingInMask
0,0,10000,1000000,2019-07-02,74,7.70,NaN,55.0,MRBx,0+0,/home/sliceruser/labels/csPCa_lesion_delineati...,/home/sliceruser/data/10000/10000_1000000_adc.mha,/home/sliceruser/data/10000/10000_1000000_cor.mha,/home/sliceruser/data/10000/10000_1000000_hbv.mha,/home/sliceruser/data/10000/10000_1000000_sag.mha,/home/sliceruser/data/10000/10000_1000000_t2w.mha,False
1,1,10001,1000001,2016-05-27,64,8.70,0.09,102.0,NaN,NaN,/home/sliceruser/labels/csPCa_lesion_delineati...,/home/sliceruser/data/10001/10001_1000001_adc.mha,/home/sliceruser/data/10001/10001_1000001_cor.mha,/home/sliceruser/data/10001/10001_1000001_hbv.mha,/home/sliceruser/data/10001/10001_1000001_sag.mha,/home/sliceruser/data/10001/10001_1000001_t2w.mha,False
2,2,10002,1000002,2021-04-18,58,4.20,0.06,74.0,NaN,NaN,/home/sliceruser/labels/csPCa_lesion_delineati...,/home/sliceruser/data/10002/10002_1000002_adc.mha,/home/sliceruser/data/10002/10002_1000002_cor.mha,/home/sliceruser/data/10002/10002_1000002_hbv.mha,/home/sliceruser/data/10002/10002_1000002_sag.mha,/home/sliceruser/data/10002/10002_1000002_t2w.mha,False
3,3,10003,1000003,2019-04-05,72,13.00,NaN,71.5,SysBx,0+0,/home/sliceruser/labels/csPCa_lesion_delineati...,/home/sliceruser/data/10003/10003_1000003_adc.mha,/home/sliceruser/data/10003/10003_1000003_cor.mha,/home/sliceruser/data/10003/10003_1000003_hbv.mha,/home/sliceruser/data/10003/10003_1000003_sag.mha,/home/sliceruser/data/10003/10003_1000003_t2w.mha,False
4,4,10004,1000004,2020-10-21,67,8.00,0.10,78.0,SysBx+MRBx,"0+0,0+0",/home/sliceruser/labels/csPCa_lesion_delineati...,/home/sliceruser/data/10004/10004_1000004_adc.mha,/home/sliceruser/data/10004/10004_1000004_cor.mha,/home/sliceruser/data/10004/10004_1000004_hbv.mha,/home/sliceruser/data/10004/10004_1000004_sag.mha,/home/sliceruser/data/10004/10004_1000004_t2w.mha,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,1495,11471,1001495,2012-08-25,71,12.50,0.21,62.0,MRBx,"3+4,N/A,3+3",/home/sliceruser/labels/csPCa_lesion_delineati...,/home/sliceruser/data/11471/11471_1001495_adc.mha,/home/sliceruser/data/11471/11471_1001495_cor.mha,/home/sliceruser/data/11471/11471_1001495_hbv.mha,/home/sliceruser/data/11471/11471_1001495_sag.mha,/home/sliceruser/data/11471/11471_1001495_t2w.mha,False
1496,1496,11472,1001496,2019-06-28,81,5.28,0.12,44.0,SysBx+MRBx,3+4,,/home/sliceruser/data/11472/11472_1001496_adc.mha,/home/sliceruser/data/11472/11472_1001496_cor.mha,/home/sliceruser/data/11472/11472_1001496_hbv.mha,/home/sliceruser/data/11472/11472_1001496_sag.mha,/home/sliceruser/data/11472/11472_1001496_t2w.mha,False
1497,1497,11473,1001497,2017-09-24,56,29.60,0.34,87.0,MRBx,0+0,/home/sliceruser/labels/csPCa_lesion_delineati...,/home/sliceruser/data/11473/11473_1001497_adc.mha,/home/sliceruser/data/11473/11473_1001497_cor.mha,/home/sliceruser/data/11473/11473_1001497_hbv.mha,/home/sliceruser/data/11473/11473_1001497_sag.mha,/home/sliceruser/data/11473/11473_1001497_t2w.mha,False
1498,1498,11474,1001498,2016-05-03,75,12.00,NaN,83.0,MRBx,3+3,/home/sliceruser/labels/csPCa_lesion_delineati...,/home/sliceruser/data/11474/11474_1001498_adc.mha,/home/sliceruser/data/11474/11474_1001498_cor.mha,/home/sliceruser/data/11474/11474_1001498_hbv.mha,/home/sliceruser/data/11474/11474_1001498_sag.mha,/home/sliceruser/data/11474/11474_1001498_t2w.mha,False


In [12]:
row = list(df.iterrows())[3]
print(row[1]['reSampledPath'])
row

/home/sliceruser/labels/csPCa_lesion_delineations/human_expert/resampled/10368_1000374.nii.gz


(3,
 Unnamed: 0                                                         3
 patient_id                                                     10003
 study_id                                                     1000003
 mri_date                                                  2019-04-05
 patient_age                                                       72
 psa                                                             13.0
 psad                                                             NaN
 prostate_volume                                                 71.5
 histopath_type                                                 SysBx
 lesion_GS                                                        0+0
 reSampledPath      /home/sliceruser/labels/csPCa_lesion_delineati...
 adc                /home/sliceruser/data/10003/10003_1000003_adc.mha
 cor                /home/sliceruser/data/10003/10003_1000003_cor.mha
 hbv                /home/sliceruser/data/10003/10003_1000003_hbv.mha
 sag            